In [11]:
import praw 
import pandas as pd

In [35]:
reddit = praw.Reddit(
    client_id="cXb7-vX9BLku6yVl6aKagg",
    client_secret="Pa1gMfMKKUqS_hyFllKzzY6d7__JPw",
    user_agent="jellyfish",
    user_name="jellyfish5544",
    password="pjy530150"
)

In [173]:
posts=pd.read_csv('suicide.csv')
posts=posts.loc[(posts.key_words=='suicide') ]    #Only analyze 'suicide', narrow the range.
posts=posts.sort_values(by='score', ascending=False)  #Score means upvoting numbers
posts_sub=posts.loc[posts.subreddit=='SuicideWatch'].sort_values(by='score', ascending=False)
posts_sub
posts

,Unnamed: 0,title,score,id,subreddit,url,num_comments,body,created,key_words
5399,5400,to block the German autobahn,79603,zdc3sl,therewasanattempt,https://v.redd.it/zie3h567h54a1,4152,NaN,1.670259e+09,suicide
1614,1614,Gabby Petito Confirmed Dead by her Father on T...,75330,prift6,awfuleverything,https://i.redd.it/m3cc2ot6ijo71.png,11510,NaN,1.632092e+09,suicide
5884,5885,"Print("" suicidal thoughts"")",74839,e4golj,dankmemes,https://i.redd.it/4r4iepqfj0241.jpg,405,NaN,1.575202e+09,suicide
5376,5377,to make his death look like a suicide,61442,gd0bzc,therewasanattempt,https://i.redd.it/ujxthsw0umw41.jpg,832,NaN,1.588548e+09,suicide
5371,5372,to stop suicide,59371,iu8k8c,therewasanattempt,https://i.redd.it/2ez8ce09sln51.jpg,1212,NaN,1.600303e+09,suicide
...,...,...,...,...,...,...,...,...,...,...
107,107,suicide,0,16ta8aa,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,3,i really want to die can someone tell me if ta...,1.695787e+09,suicide
2462,2462,suicide by drowning? pros and cons?,0,rflftv,lonely,https://www.reddit.com/r/lonely/comments/rflft...,15,need help with that.. plz dm,1.639418e+09,suicide
1721,1721,Person wishes suicide and rape on someone who ...,0,hl65r8,awfuleverything,https://i.redd.it/cew0g2ezav851.jpg,6,NaN,1.593880e+09,suicide
2504,2504,I want to make a fake suicide note just to see...,0,z6ddon,lonely,https://www.reddit.com/r/lonely/comments/z6ddo...,3,I bet all the people who ignore my calls to ha...,1.669585e+09,suicide


In [37]:
import nltk                                # Python library for NLP
from nltk.corpus import twitter_samples    # sample Twitter dataset from NLTK
import matplotlib.pyplot as plt            # library for visualization
import random 
import numpy as np
nltk.download('twitter_samples')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     /Users/panjiayi/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


True

In [3]:
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [4]:
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [5]:
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem import PorterStemmer        # module for stemming
from nltk.tokenize import TweetTokenizer 

In [6]:
def process_tweet(tweet):
    tweet2 = re.sub(r'^RT[\s]+', '', tweet)
    tweet2 = re.sub(r'https?://[^\s\n\r]+', '', tweet2)
    tweet2 = re.sub(r'#', '', tweet2)
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet2)
    stopwords_english = stopwords.words('english')
    tweets_clean = []
    for word in tweet_tokens: # Go through every word in your tokens list
        if (word not in stopwords_english and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            tweets_clean.append(word)
    stemmer = PorterStemmer() 
    # Create an empty list to store the stems
    tweets_stem = [] 
    for word in tweets_clean:
        stem_word = stemmer.stem(word)  # stemming word
        tweets_stem.append(stem_word)  # append to the list
    return tweets_stem

def build_freqs(tweets, ys):
    """Build frequencies.
    Input:
        tweets: a list of tweets
        ys: an m x 1 array with the sentiment label of each tweet
            (either 0 or 1)
    Output:
        freqs: a dictionary mapping each (word, sentiment) pair to its
        frequency
    """
    # Convert np array to list since zip needs an iterable.
    # The squeeze is necessary or the list ends up with one element.
    # Also note that this is just a NOP if ys is already a list.
    yslist = np.squeeze(ys).tolist()

    # Start with an empty dictionary and populate it by looping over all tweets
    # and over all processed words in each tweet.
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in process_tweet(tweet):
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1    
    return freqs

freqs = build_freqs(train_x, train_y)  #Build Frequancy

def sigmoid(z): 
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''
    
    ### START CODE HERE ###
    # calculate the sigmoid of z
    h = 1/(1+np.exp(-z))
    ### END CODE HERE ###
    
    return h

def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    Hint: you might want to print the cost to make sure that it is going down.
    '''
    ### START CODE HERE ###
    # get 'm', the number of rows in matrix x
    m = x.shape[0]
    for i in range(0, num_iters):
        # get z, the dot product of x and theta
        z = x@theta
        # get the sigmoid of z
        h = sigmoid(z)   #python自变量是标量还是向量无影响
        # calculate the cost function
        J = (-1/m)*(np.transpose(y)@np.log(h)+np.transpose((1-y))@np.log(1-h))  #向量运算要用@
        # update the weights theta
        theta = theta-alpha/m*(np.transpose(x)@(h-y))
        
    ### END CODE HERE ###
    J = float(J)
    return J,theta


def extract_features(tweet, freqs, process_tweet=process_tweet):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements for [bias, positive, negative] counts
    x = np.zeros(3) 
    
    # bias term is set to 1
    x[0] = 1 
    
    ### START CODE HERE ###
    
    # loop through each word in the list of words
    for word in word_l:
        if (word, 1) in freqs:
            # increment the word count for the positive label 1
            x[1] += freqs[(word,1)]
        if (word, 0) in freqs:
            # increment the word count for the negative label 0
            x[2] += freqs[(word,0)]
        
    ### END CODE HERE ###
    
    x = x[np.newaxis, :]  # adding batch dimension for further processing
    assert(x.shape == (1, 3))
    return x



In [7]:
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)

/var/folders/ss/sv50jmf54z5fg7h09y4ws6gc0000gn/T/ipykernel_11621/2023382314.py:93: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


In [8]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs, process_tweet=process_tweet)
    
    # make the prediction using x and theta
    y_pred = sigmoid(x@theta)
    
    ### END CODE HERE ###
    
    return y_pred

In [9]:
def test_logistic_regression(test_x, test_y, freqs, theta, predict_tweet=predict_tweet):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### START CODE HERE ###
    
    # the list for storing predictions
    y_hat = list()
    
    for tweet in test_x:
        # get the label prediction for the tweet
        z=extract_features(tweet, freqs, process_tweet=process_tweet)@theta
        y_pred = sigmoid(z)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.0)
        else:
            # append 0 to the list
            y_hat.append(0.0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    accuracy =np.sum(y_hat==test_y.flatten())/len(test_x)

    ### END CODE HERE ###
    
    return accuracy

In [10]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9950


In [135]:
#0  iloc[4]:"to stop suicide"
submission = reddit.submission(url="https://www.reddit.com/r/therewasanattempt/comments/iu8k8c/to_stop_suicide/")   #Find the first submission in 'suicide.csv' document 
                                                     
submission.comments.replace_more(limit=None)  
id=[]
for comment in submission.comments.list():
    id.append(comment.author)   #Find the author of comments of first submission
    
filtered_id = [item for item in id if item is not None]   #Some author log off so I cleared them.
print(filtered_id[0:10])    #We are going to analyze 10 authors who were influenced by the first submission

[Redditor(name='coderedninja'), Redditor(name='RedMirricat'), Redditor(name='hadToPutDickinCatHmm'), Redditor(name='Defy19'), Redditor(name='yennifluent'), Redditor(name='yeahwellokay'), Redditor(name='thistyviolin'), Redditor(name='ViciousNakedMoleRat'), Redditor(name='gsychopato'), Redditor(name='Deja__Vu__')]


In [136]:
DIF0=[]
for i in [0,2,3,4,5,6,7,8,9,10,11,12,13]:  #Total 13 redditors who made comments
    comment0=[]
    for submission in reddit.redditor(filtered_id[i].name).submissions.new(limit=200):
        comment0.append([submission.title, 
                         submission.created,
                         i])
    comment0 = pd.DataFrame(comment0,
                            columns=['title','created','Serial Number'])
    #comment0 = comment0.sort_values(by='created', ascending=True)
    split_time=posts['created'].iloc[4]
    comment0_before=comment0.loc[comment0['created']<split_time]
    comment0_after=comment0.loc[comment0['created']>split_time]
    text0_before = ''.join(comment0_before.title.values)
    y0_before=predict_tweet(text0_before, freqs, theta)
    text0_after = ''.join(comment0_after.title.values)
    y0_after=predict_tweet(text0_after, freqs, theta)
    difference0=(y0_before-y0_after)
    DIF0.append(difference0)
    print("Positiveness before the first submission is",y0_before,"and after the first submission is",y0_after,".\n","The difference is", difference0)

Positiveness before the first submission is [[0.4969238]] and after the first submission is [[0.48817875]] .
 The difference is [[0.00874505]]
Positiveness before the first submission is [[0.44939618]] and after the first submission is [[0.51828169]] .
 The difference is [[-0.06888552]]
Positiveness before the first submission is [[0.50000002]] and after the first submission is [[0.45621244]] .
 The difference is [[0.04378758]]
Positiveness before the first submission is [[0.47572599]] and after the first submission is [[0.48828321]] .
 The difference is [[-0.01255722]]
Positiveness before the first submission is [[0.35084829]] and after the first submission is [[0.32712443]] .
 The difference is [[0.02372387]]
Positiveness before the first submission is [[0.50000002]] and after the first submission is [[0.49901796]] .
 The difference is [[0.00098205]]
Positiveness before the first submission is [[0.52956741]] and after the first submission is [[0.52122144]] .
 The difference is [[0.00

In [137]:
DIF0

[array([[0.00874505]]),
 array([[-0.06888552]]),
 array([[0.04378758]]),
 array([[-0.01255722]]),
 array([[0.02372387]]),
 array([[0.00098205]]),
 array([[0.00834597]]),
 array([[0.03022548]]),
 array([[0.00179517]]),
 array([[0.02410308]]),
 array([[-0.03282989]]),
 array([[-0.01167864]]),
 array([[-0.38896801]])]

In [168]:
#1  iloc[20]:"to prevent suicide"
submission = reddit.submission(url="https://www.reddit.com/r/therewasanattempt/comments/16psiru/to_prevent_suicide/")   #Find the second submission in 'suicide.csv' document 
                        
submission.comments.replace_more(limit=None)  
id=[]
for comment in submission.comments.list():
    id.append(comment.author)   #Find the author of comments of second submission
    
filtered_id = [item for item in id if item is not None]   #Some author log off so I cleared them.
print(filtered_id[0:10])    #We are going to analyze 10 authors who were influenced by the second submission
DIF1=[]
for i in [0,1,2,3,4,5,6,7,8,9,11,12,13]:  #Total 13 redditors who made comments
    comment1=[]
    for submission in reddit.redditor(filtered_id[i].name).submissions.new(limit=200):
        comment1.append([submission.title, 
                         submission.created,
                         i])
    comment1 = pd.DataFrame(comment1,
                            columns=['title','created','Serial Number'])
    #comment1 = comment1.sort_values(by='created', ascending=True)
    split_time=posts['created'].iloc[20]
    comment1_before=comment1.loc[comment1['created']<split_time]
    comment1_after=comment1.loc[comment1['created']>split_time]
    text1_before = ''.join(comment1_before.title.values)
    y1_before=predict_tweet(text1_before, freqs, theta)
    text1_after = ''.join(comment1_after.title.values)
    y1_after=predict_tweet(text1_after, freqs, theta)
    difference1=(y1_before-y1_after)
    DIF1.append(difference1)
    print("Positiveness before the second submission is",y1_before,"and after the second submission is",y1_after,".\n","The difference is", difference1)


[Redditor(name='AutoModerator'), Redditor(name='DMurBOOBS-I-Dare-You'), Redditor(name='snaeper'), Redditor(name='lazemachine'), Redditor(name='RahnKavall'), Redditor(name='AtLastThe1971Show'), Redditor(name='MileHighOllie'), Redditor(name='TH0T_P0L1CE'), Redditor(name='Harbor_Barber'), Redditor(name='magaphone12')]
Positiveness before the second submission is [[0.50000002]] and after the second submission is [[0.29046203]] .
 The difference is [[0.20953799]]
Positiveness before the second submission is [[0.41145243]] and after the second submission is [[0.53330328]] .
 The difference is [[-0.12185085]]
Positiveness before the second submission is [[0.37263469]] and after the second submission is [[0.50390841]] .
 The difference is [[-0.13127373]]
Positiveness before the second submission is [[0.50099671]] and after the second submission is [[0.51927504]] .
 The difference is [[-0.01827833]]
Positiveness before the second submission is [[0.39768925]] and after the second submission is [

In [169]:
DIF1

[array([[0.20953799]]),
 array([[-0.12185085]]),
 array([[-0.13127373]]),
 array([[-0.01827833]]),
 array([[-0.10143077]]),
 array([[-0.0047149]]),
 array([[0.00104274]]),
 array([[0.]]),
 array([[0.08352825]]),
 array([[-0.01878983]]),
 array([[0.41824687]]),
 array([[0.]]),
 array([[0.21633767]])]

In [164]:
#2  loc[47]:"'If you were actually suicidal you'd have killed yourself already and not tell anybody' FUCK OFFFFFFF"
submission = reddit.submission(url="https://www.reddit.com/r/SuicideWatch/comments/jnrz57/if_you_were_actually_suicidal_youd_have_killed/")   #Find the third submission in 'suicide.csv' document 
            
submission.comments.replace_more(limit=None)  
id=[]
for comment in submission.comments.list():
    id.append(comment.author)   #Find the author of comments of third submission
    
filtered_id = [item for item in id if item is not None]   #Some author log off so I cleared them.
print(filtered_id[0:10])    
DIF2=[]
for i in [0,1,2,3,4,5,6,7,8,9,11,12,13]:  #Total 13 redditors who made comments
    comment2=[]
    for submission in reddit.redditor(filtered_id[i].name).submissions.new(limit=200):
        comment2.append([submission.title, 
                         submission.created,
                         i])
    comment2 = pd.DataFrame(comment2,
                            columns=['title','created','Serial Number'])
    #comment2 = comment2.sort_values(by='created', ascending=True)
    split_time=posts_sub['created'].loc[47]
    comment2_before=comment2.loc[comment2['created']<split_time]
    comment2_after=comment2.loc[comment2['created']>split_time]
    text2_before = ''.join(comment2_before.title.values)
    y2_before=predict_tweet(text2_before, freqs, theta)
    text2_after = ''.join(comment2_after.title.values)
    y2_after=predict_tweet(text2_after, freqs, theta)
    difference2=(y2_before-y2_after)
    DIF2.append(difference2)
    print("Positiveness before the third submission is",y2_before,"and after the third submission is",y2_after,".\n","The difference is", difference2)

[Redditor(name='Throwawayrae20'), Redditor(name='Smearmytables'), Redditor(name='ToyboxOfThoughts'), Redditor(name='BloodyCattie'), Redditor(name='Stalanium'), Redditor(name='BeastKingYt'), Redditor(name='otto_vanoss'), Redditor(name='Ribichka'), Redditor(name='realityhofosho'), Redditor(name='Qfanjen')]
Positiveness before the third submission is [[0.49985541]] and after the third submission is [[0.49753015]] .
 The difference is [[0.00232526]]
Positiveness before the third submission is [[0.61950732]] and after the third submission is [[0.43983531]] .
 The difference is [[0.17967201]]
Positiveness before the third submission is [[0.50000002]] and after the third submission is [[0.45941136]] .
 The difference is [[0.04058866]]
Positiveness before the third submission is [[0.46783644]] and after the third submission is [[0.46929102]] .
 The difference is [[-0.00145458]]
Positiveness before the third submission is [[0.45247118]] and after the third submission is [[0.44479762]] .
 The di

In [165]:
DIF2

[array([[0.00232526]]),
 array([[0.17967201]]),
 array([[0.04058866]]),
 array([[-0.00145458]]),
 array([[0.00767356]]),
 array([[0.15068374]]),
 array([[-0.04957379]]),
 array([[0.13555795]]),
 array([[0.15920085]]),
 array([[-0.05704362]]),
 array([[-0.14351555]]),
 array([[-0.05567369]]),
 array([[0.1604567]])]

In [171]:
#3  loc[3]:"I'm a fully-functioning suicidal."
submission = reddit.submission(url="https://www.reddit.com/r/SuicideWatch/comments/ppjowo/im_a_fullyfunctioning_suicidal/")   #Find the fourth submission in 'suicide.csv' document 
                                                     
submission.comments.replace_more(limit=None)  
id=[]
for comment in submission.comments.list():
    id.append(comment.author)   #Find the author of comments of fourth submission
    
filtered_id = [item for item in id if item is not None]   #Some author log off so I cleared them.
print(filtered_id[0:10])   
DIF3=[]
for i in [0,1,2,3,4,6,7,8,9,10,11,12,13]:  #Total 33 redditors who made comments
    comment3=[]
    for submission in reddit.redditor(filtered_id[i].name).submissions.new(limit=200):
        comment3.append([submission.title, 
                         submission.created,
                         i])
    comment3 = pd.DataFrame(comment3,
                            columns=['title','created','Serial Number'])
    #comment3 = comment3.sort_values(by='created', ascending=True)
    split_time=posts_sub['created'].loc[3]
    comment3_before=comment3.loc[comment3['created']<split_time]
    comment3_after=comment3.loc[comment3['created']>split_time]
    text3_before = ''.join(comment3_before.title.values)
    y3_before=predict_tweet(text3_before, freqs, theta)
    text3_after = ''.join(comment3_after.title.values)
    y3_after=predict_tweet(text3_after, freqs, theta)
    difference3=(y3_before-y3_after)
    DIF3.append(difference3)
    print("Positiveness before the fourth submission is",y3_before,"and after the fourth submission is",y3_after,".\n","The difference is", difference3)

[Redditor(name='MoosieGoose'), Redditor(name='JollyK9'), Redditor(name='Peaceamongw0rlds'), Redditor(name='Southern_Ad3032'), Redditor(name='bduwowy272habbw'), Redditor(name='WeissFan43'), Redditor(name='Late_Introduction203'), Redditor(name='kapster68'), Redditor(name='TheApertureMonkey'), Redditor(name='I_hate_me_lol')]
Positiveness before the fourth submission is [[0.50167878]] and after the fourth submission is [[0.50678963]] .
 The difference is [[-0.00511085]]
Positiveness before the fourth submission is [[0.37987739]] and after the fourth submission is [[0.46399299]] .
 The difference is [[-0.08411561]]
Positiveness before the fourth submission is [[0.47743997]] and after the fourth submission is [[0.50000002]] .
 The difference is [[-0.02256005]]
Positiveness before the fourth submission is [[0.49537092]] and after the fourth submission is [[0.00552542]] .
 The difference is [[0.4898455]]
Positiveness before the fourth submission is [[0.48475134]] and after the fourth submissio

In [172]:
DIF3

[array([[-0.00511085]]),
 array([[-0.08411561]]),
 array([[-0.02256005]]),
 array([[0.4898455]]),
 array([[0.04525343]]),
 array([[0.]]),
 array([[0.]]),
 array([[-0.01261079]]),
 array([[-0.17526799]]),
 array([[0.00952793]]),
 array([[-0.02056604]]),
 array([[-0.00378704]]),
 array([[0.01589604]])]